<a href="https://colab.research.google.com/github/cksdud7007/NHQV_text_classification/blob/master/soynlp_mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0,nb_path)

Mounted at /content/mnt



$$
\text{cohesion}(n) = \left( \prod_{i=1}^{n-1} P(c_{1:i+1} | c_{1:i}) \right)^{\frac{1}{n-1}}
$$




* $cohesion(2) = P(노란 | 노)$
* $cohesion(3) = \sqrt[2]{\left( P(노란 | 노) \cdot P(노란색 | 노란) \right)}$

# Soynlp로 먼저 말뭉치 만들어서 학습시킨 후 명사 추출하기


In [ ]:
# soynlp 설치
!pip install soynlp

     |████████████████████████████████| 419kB 8.9MB/s 


In [ ]:
# 학습시킬 파일 불러오기
with open('/content/mnt/Shareddrives/Real_News/찬영/corpus.txt','r',encoding='utf-8') as file:
    text = file.readlines()

In [ ]:
text


['[이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도\n',
 '"실적기반" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개\n',
 '하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시\n',
 '종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지\n',
 '전국적인 소비 붐 조성에 기여할 예정\n',
 '[이데일리 권오석 기자] 중소벤처기업부(이하 중기부)는 대한민국 동행세일에 7개 TV홈쇼핑사가 홍보와 판매에 동참한다고 26일 밝혔다\n',
 '대한민국 동행세일은 라이브 커머스, 언택트 콘서트, O2O 행사 연계 등 비대면이라는 새로운 형태의 소비촉진 행사다\n',
 '6개 권역에서의 현장행사와 온·오프라인 판촉, TV홈쇼핑 등 연계행사를 통해 소비심리를 진작하기 위함이다\n',
 '이번 동행세일에서는 롯데·공영·CJ·현대·GS·NS·홈앤쇼핑 등 7개 TV 홈쇼핑사가 동행세일의 홍보와 방송 판매에 동참해 전국적인 소비 붐 조성에 기여한다\n',
 '동행세일 기간 중 홈쇼핑사에서는 판매방송 사이에 영상을 노출하는 방식(SB, station break)을 통해 행사 취지와 메시지를 홍보한다\n',
 '첫 방송으로 6월 26일부터 공영홈쇼핑은 마스크, 식기세트 등 생활용품과 신선식품 등의 17개 제품을 최대 50% 할인된 가격으로 판매하고 이어서 6개 민간 홈쇼핑사들도 동행세일 판매방송에 동참할 계획이다\n',
 '특히 롯데홈쇼핑은 6월 26일 부산 벡스코에서 현장 이원 생방송을 통해 동행세일 현장을 국민에게 생생하게 전달했다\n',
 '박영선 중기부 장관이 쇼호스트로 깜짝 등장해 동행세일을 마련한 취지와 대한민국 중소기업 발전을 위한 앞으로의 계획 등을 밝히기도 했다\n',
 '현장 이원 생방송의 다음 주자로는, 공영홈쇼핑이 7월 10일부터 12일까지 서울 코엑스에서 이원 현장 생방송을 통해 동행세일의 마지막을 장식한다\n',
 '박영선 장관은 이번 동행세일 행사에 TV홈쇼핑사의 동참을 통해 내수 활성화에 한

In [ ]:
# 학습시키기 (가짜뉴스와 진짜뉴스에 있는 명사를 잘 추출해내기 위함 -> mecab사전에 잘 정의되지 않는 명사들이 존재하기 때문)
from soynlp.utils import DoublespaceLineCorpus
from soynlp.noun import LRNounExtractor_v2


noun_extractor = LRNounExtractor_v2(verbose=True)
nouns = noun_extractor.train_extract(text)

[Noun Extractor] use default predictors
[Noun Extractor] num features: pos=3929, neg=2321, common=107
[Noun Extractor] counting eojeols
[EojeolCounter] n eojeol = 646153 from 291401 sents. mem=0.334 Gb                    
[Noun Extractor] complete eojeol counter -> lr graph
[Noun Extractor] has been trained. #eojeols=7423994, mem=1.656 Gb
[Noun Extractor] batch prediction was completed for 185759 words
[Noun Extractor] checked compounds. discovered 119085 compounds
[Noun Extractor] postprocessing detaching_features : 170279 -> 143664
[Noun Extractor] postprocessing ignore_features : 143664 -> 143420
[Noun Extractor] postprocessing ignore_NJ : 143420 -> 141586
[Noun Extractor] 141586 nouns (119085 compounds) with min frequency=1
[Noun Extractor] flushing was done. mem=2.078 Gb                    
[Noun Extractor] 77.65 % eojeols are covered


In [ ]:
# 학습시킨 nouns파일로 명사를 추출해보기
tgt_noun = []
for noun in nouns:
    if (nouns[noun][0] >= 40) and (nouns[noun][1]) >= 0.5: # nouns[noun][0]은 단어의 빈도 수를 나타내고 nouns[noun][1]은 단어의 score를 나타냄 => 파라미터를 어떻게 설정해주냐에 따라 단어가 잘 추출됨을 의미
        if len(noun) == 3:   # 일단 예시로 명사길이가 세글자인 것만 추출
            tgt_noun.append(noun)

In [ ]:
tgt_noun

In [ ]:
import pandas as pd
import re

In [ ]:
# 그래서 끝에 잘못 딸린 단어들을 다시 정제해주기로 함.
non_noun_eomi_list = ('.', ',', '(',')','들','지')
tgt_noun = set([item.strip(""""',.‘“[]()”’ """) for item in tgt_noun if (item[-1] not in non_noun_eomi_list) \
                        & (sum([1 if char.isdigit() else 0 for char in item]) <= 3) \
                        & (len(re.sub('[^가-힣]', '', item)) <= 3) \
                        & (not re.search('[^가-힣0-9]', item))])

In [ ]:
# 정제된 명사 (데이터 프레임)
pd.DataFrame(tgt_noun,columns= ['noun'])

In [ ]:
# 데이터프레임으로 만든 명사를 csv파일로 저장 (경로는 사용자지정:일단 만남의 광장으로 지정)
pd.DataFrame(tgt_noun,columns= ['noun']).to_csv('/content/mnt/Shareddrives/Real_News/만남의 광장/noun.csv', encoding = 'ANSI')

# Soynlp로 만든 명사들로 Mecab 사전 추가해주기

In [ ]:
import pandas as pd

In [ ]:
# 파일 불러오기
mecab_dict = pd.read_csv('/content/mnt/Shareddrives/Real_News/찬영/noun.csv',encoding='cp949')

In [ ]:
# 먼저 mecab 설치하기
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 75 (delta 33), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [ ]:
cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [ ]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 8.7MB/s 
     |████████████████████████████████| 460kB 51.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-12-26 07:36:15--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c5:2ef4, 2406:da00:ff00::6b17:d1f5, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloa

In [ ]:
# mecab 불러오기
from konlpy.tag import Mecab

#####사전 추가하기 전 사전작업 (설치)

In [ ]:
cd /content/mecab-ko-dic-2.1.1-20180720

/content/mecab-ko-dic-2.1.1-20180720


In [ ]:
!pip install jamo

In [ ]:
# 종성 여부 함수 정의
from jamo import h2j, j2hcj 

def get_jongsung_TF(sample_text):
   sample_text_list = list(sample_text) 
   last_word = sample_text_list[-1] 
   last_word_jamo_list = list(j2hcj(h2j(last_word))) 
   last_jamo = last_word_jamo_list[-1] 
   jongsung_TF = "T" 

   if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
     jongsung_TF = "F" 
   return jongsung_TF


In [ ]:
# mecab_dict 정의 

with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:  # 여러 파일들이 있는데 저는 이 파일에 넣어줌.
  file_data = f.readlines()

word_list = mecab_dict['0'].to_list()  # column명이 0으로 되어있어서 0으로 넣어줌

# line 부분에 0,1,2는 우선순위를 넣어주기 위함(숫자가 낮아질수록 가장먼저 토큰화 됨.)
for word in word_list:
  jongsung_TF = get_jongsung_TF(word)
  if len(word)>=4:
    line = '{},,,0,NNP,*,{},{},*,*,*,*,*\n'.format(word , jongsung_TF, word)
    file_data.append(line)
    
  elif len(word) == 3:
    line = '{},,,1,NNP,*,{},{},*,*,*,*,*\n'.format(word , jongsung_TF, word)
    file_data.append(line)

  elif len(word) == 2:
    line = '{},,,2,NNP,*,{},{},*,*,*,*,*\n'.format(word , jongsung_TF, word)
    file_data.append(line)


In [ ]:
# 잘 정의되었는지 보기

with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f:
  for line in file_data:
    f.write(line)

##### 사전 추가를 한 후 다시 재업로드 해줘야 됨. => 그래야 사전 추가된 인식 가능

In [ ]:
ls tools

add-userdic.sh*  convert_for_using_store.sh*  mecab-bestn.sh*


In [ ]:
!bash ./tools/add-userdic.sh

generating userdic...
nnp.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/nnp.csv ... 
done!
person.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/person.csv ... 
done!
place.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/place.csv ... 
done!
test -z "model.bin matrix.bin char.bin sys.dic unk.dic" || rm -f model.bin matrix.bin char.bin sys.dic unk.dic
/usr/local/libexec/mecab/mecab-dict-index -d . -o . -f UTF-8 -t UTF-8
reading ./unk.def ... 13
emitting double-array: 100% |###########################################| 
reading ./ETM.csv ... 133
reading ./NP.csv ... 342
reading ./user-nnp.csv ... 12391
reading ./NR.csv .

In [ ]:
user_nnp = pd.read_csv('/content/mecab-ko-dic-2.1.1-20180720/user-nnp.csv')
user_nnp

,대우,1786,3545,3821,NNP,*,F,대우.1,*.1,*.2,*.3,*.4,*.5
0,구글,1786,3546,2953,NNP,*,T,구글,*,*,*,*,*
1,에이프로젠,1786,3546,2953,NNP,*,T,에이프로젠,*,*,*,*,*
2,의료현장,1786,3546,2953,NNP,*,T,의료현장,*,*,*,*,*
3,목표주가,1786,3545,2953,NNP,*,F,목표주가,*,*,*,*,*
4,하이투자증권,1786,3546,2953,NNP,*,T,하이투자증권,*,*,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12385,적응증,1786,3546,2953,NNP,*,T,적응증,*,*,*,*,*
12386,부회장,1786,3546,2556,NNP,*,T,부회장,*,*,*,*,*
12387,김성주,1786,3545,2953,NNP,*,F,김성주,*,*,*,*,*
12388,교차로,1786,3545,2675,NNP,*,F,교차로,*,*,*,*,*


In [ ]:
for i in user_nnp.대우:
  if len(i)>=4:
    user_nnp.loc[(user_nnp.대우 == i),'3821'] = 0

  elif len(i) == 3:
    user_nnp.loc[(user_nnp.대우 == i),'3821'] = 1

  elif len(i) == 2:
    user_nnp.loc[(user_nnp.대우 == i),'3821'] = 2

In [ ]:
user_nnp.to_csv('/content/mecab-ko-dic-2.1.1-20180720/user-nnp.csv',index=False)

In [ ]:
!make clean

test -z "model.bin matrix.bin char.bin sys.dic unk.dic" || rm -f model.bin matrix.bin char.bin sys.dic unk.dic


In [ ]:
!make install

/usr/local/libexec/mecab/mecab-dict-index -d . -o . -f UTF-8 -t UTF-8
reading ./unk.def ... 13
emitting double-array: 100% |###########################################| 
reading ./ETM.csv ... 133
reading ./NP.csv ... 342
reading ./user-nnp.csv ... 12391
reading ./NR.csv ... 482
reading ./VCN.csv ... 7
reading ./MAG.csv ... 14242
reading ./user-person.csv ... 1
reading ./NNB.csv ... 140
reading ./Place-station.csv ... 1145
reading ./J.csv ... 416
reading ./CoinedWord.csv ... 148
reading ./EF.csv ... 1820
reading ./NNBC.csv ... 677
reading ./Wikipedia.csv ... 36762
reading ./EP.csv ... 51
reading ./MAJ.csv ... 240
reading ./Person-actor.csv ... 99230
reading ./Inflect.csv ... 44820
reading ./Place.csv ... 30303
reading ./VCP.csv ... 9
reading ./Preanalysis.csv ... 5
reading ./VV.csv ... 7331
reading ./Person.csv ... 196459
reading ./XSV.csv ... 23
reading ./IC.csv ... 1305
reading ./Group.csv ... 3176
reading ./NNP.csv ... 2371
reading ./VA.csv ... 2360
reading ./user-place.csv ... 2
rea

In [ ]:
# 이제 mecab함수를 불러와서 토큰화 진행하면 됨.
mecab=Mecab()

In [ ]:
print(mecab.pos('내일 폭등 예상되는 극비 종목 BEST 5 .  눌러서 확인하기'))

[('내일', 'NNP'), ('폭등', 'NNP'), ('예상', 'NNP'), ('되', 'VV'), ('는', 'ETM'), ('극비', 'NNG'), ('종목', 'NNP'), ('BEST', 'SL'), ('5', 'SN'), ('.', 'SF'), ('눌러서', 'VV+EC'), ('확인', 'NNG'), ('하', 'XSV'), ('기', 'ETN')]


In [ ]:
print(mecab.pos('내일 폭등 기대주  1등 종목 무료 확인하기'))

[('내일', 'NNP'), ('폭등', 'NNP'), ('기대', 'NNP'), ('주', 'NNP'), ('1등', 'NNP'), ('종목', 'NNP'), ('무료', 'NNP'), ('확인', 'NNG'), ('하', 'XSV'), ('기', 'ETN')]
